In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
node_path = '../data/processed_data/node_link/daegu_selected_nodes.csv'
link_path = '../data/processed_data/node_link/daegu_selected_links.csv'

node = pd.read_csv(node_path)
link = pd.read_csv(link_path)

In [3]:
def fixer(x):
    try: return int(x)
    except ValueError:
        return 0
    
target_file = '20230901_5Min.csv'

traffic_data_path = os.path.join('../data/raw_data/traffic', target_file)
traffic_data_col = ['Date', 'Time', 'LINK_ID', 'Speed']
traffic_data_type = {'Date':'Int32', 'Time':'Int32', 'LINK_ID':'Int64','Speed':'Float32'}

traffic_index = pd.read_csv(traffic_data_path, usecols=[2], names=['LINK_ID'], converters={2:fixer}, on_bad_lines='warn')

In [4]:
print(pd.api.types.infer_dtype(traffic_index['LINK_ID']))

integer


In [5]:
traffic_index

,LINK_ID
0,1000000100
1,1000000200
2,1000000300
3,1000000301
4,1000000302
...,...
129135078,4180383901
129135079,4180384001
129135080,4180384101
129135081,4180384201


In [6]:
link_set = set(link['LINK_ID'])
temp = traffic_index[traffic_index['LINK_ID'].map(lambda x: x not in link_set)].index

In [7]:
traffic_data = pd.read_csv(traffic_data_path, usecols=[0,1,2,4], names=traffic_data_col, dtype=traffic_data_type, skiprows=temp)

In [8]:
traffic_data

,Date,Time,LINK_ID,Speed
0,20230901,0,1500000100,53.0
1,20230901,0,1500000200,26.0
2,20230901,0,1500000506,18.0
3,20230901,0,1500000605,48.0
4,20230901,0,1500000700,41.0
...,...,...,...,...
108415,20230901,2355,1550382600,47.0
108416,20230901,2355,1550383300,80.0
108417,20230901,2355,1550383400,21.0
108418,20230901,2355,1550383500,18.0


In [9]:
traffic_data = traffic_data[['Time', 'LINK_ID', 'Speed']].set_index(['LINK_ID', 'Time']).unstack(level=0)
traffic_data.columns = [x[1] for x in traffic_data.columns.values]
traffic_data = traffic_data.sort_index()
traffic_data

,1500000100,1500000200,1500000506,1500000605,1500000700,1500000800,1500001100,1500001200,1500003704,1500003706,...,1550382100,1550382200,1550382300,1550382400,1550382500,1550382600,1550383300,1550383400,1550383500,1550383600
Time,,,,,,,,,,,,,,,,,,,,,
0,53.0,26.0,18.0,48.0,41.0,43.0,28.0,35.0,41.0,24.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
5,31.0,28.0,19.0,41.0,41.0,41.0,27.0,29.0,28.0,26.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
10,21.0,31.0,28.0,41.0,24.0,53.0,36.0,31.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
15,24.0,38.0,21.0,43.0,26.0,53.0,39.0,36.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
20,26.0,40.0,36.0,43.0,39.0,53.0,36.0,36.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2325,18.0,33.0,29.0,41.0,34.0,30.0,27.0,34.0,33.0,39.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0
2335,34.0,30.0,21.0,41.0,36.0,29.0,27.0,34.0,28.0,24.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0
2345,33.0,30.299999,20.0,41.0,34.0,34.0,28.0,30.5,28.0,48.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0


# Sanity check

In [10]:
# fill nan to 0
print(traffic_data.isna().sum().sum())
if traffic_data.isna().sum().sum() > 0:
    print('Fill na to 0')
    traffic_data.fillna(value=0)
print(traffic_data.isna().sum().sum())

0
0


In [11]:
# column check
(link['LINK_ID'] == traffic_data.columns.values).sum() == len(link['LINK_ID'])

True

# linear interpolation

In [12]:
l = len(traffic_data.index)
prev = traffic_data.index[0]
append_list = []
idx_list = []
pos = 1

while pos < l:
    cur = traffic_data.index[pos]
    gap = (cur//100-prev//100)*60+cur%100-prev%100
    for i in range(5, gap, 5):
        prev += 5
        if prev % 100 == 60:
            prev += 40
        append_list.append(((gap-i)*traffic_data.iloc[pos-1]+i*traffic_data.iloc[pos])/gap)
        idx_list.append(prev)
    pos += 1
    prev = cur

In [13]:
idx_list

[110,
 120,
 130,
 140,
 210,
 220,
 230,
 240,
 310,
 320,
 330,
 340,
 410,
 420,
 430,
 440,
 610,
 620,
 630,
 640,
 650,
 710,
 720,
 730,
 740,
 750,
 810,
 820,
 830,
 835,
 840,
 850,
 910,
 920,
 930,
 940,
 950,
 1010,
 1020,
 1030,
 1040,
 1110,
 1120,
 1130,
 1140,
 1210,
 1220,
 1230,
 1240,
 1310,
 1320,
 1330,
 1340,
 1410,
 1420,
 1430,
 1440,
 1610,
 1620,
 1630,
 1640,
 1650,
 1710,
 1720,
 1730,
 1740,
 1750,
 1810,
 1820,
 1830,
 1840,
 1850,
 1910,
 1920,
 1930,
 1940,
 1950,
 2010,
 2020,
 2030,
 2040,
 2110,
 2120,
 2130,
 2140,
 2210,
 2220,
 2230,
 2240,
 2310,
 2320,
 2330,
 2340]

In [14]:
traffic_data = pd.concat([traffic_data, pd.DataFrame(append_list, index=idx_list)]).sort_index()

In [15]:
traffic_data

,1500000100,1500000200,1500000506,1500000605,1500000700,1500000800,1500001100,1500001200,1500003704,1500003706,...,1550382100,1550382200,1550382300,1550382400,1550382500,1550382600,1550383300,1550383400,1550383500,1550383600
0,53.0,26.0,18.0,48.0,41.0,43.0,28.0,35.0,41.0,24.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
5,31.0,28.0,19.0,41.0,41.0,41.0,27.0,29.0,28.0,26.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
10,21.0,31.0,28.0,41.0,24.0,53.0,36.0,31.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
15,24.0,38.0,21.0,43.0,26.0,53.0,39.0,36.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
20,26.0,40.0,36.0,43.0,39.0,53.0,36.0,36.0,28.0,28.0,...,13.0,15.0,33.0,21.5,16.0,47.0,19.0,21.0,18.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,34.0,30.0,21.0,41.0,36.0,29.0,27.0,34.0,28.0,24.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0
2340,33.5,30.15,20.5,41.0,35.0,31.5,27.5,32.25,28.0,36.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0
2345,33.0,30.299999,20.0,41.0,34.0,34.0,28.0,30.5,28.0,48.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0
2350,24.0,32.0,23.0,32.0,34.0,39.0,28.0,33.0,31.0,48.0,...,6.0,10.0,33.0,23.0,16.0,47.0,80.0,21.0,18.0,26.0


# Save

In [ ]:
traffic_data.to_csv(os.path.join('../data/processed_data/traffic', 'p_'+target_file), index=False) 